## Step 1: Create table

In [ ]:
%%sql
CREATE TABLE `farfetch_listings` (
  `brand_name` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `gender` varchar(50) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `image_cutout_url` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `image_model_url` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
  `price` decimal(10,2) DEFAULT NULL,
  `short_description` text CHARACTER SET utf8 COLLATE utf8_general_ci,
  SORT KEY `price` (`price`),
  FULLTEXT KEY `description_fulltext` (`short_description`)
)

In [ ]:
%%sql
show tables;

## Step 2: Ingest data from S3

In [ ]:
%%sql
-- since the bucket is open, you can leave the credentials clause as it is
create or replace pipeline `farfetch_pipeline` as
load data S3 's3://style-snatcher-demo/farfetch_listings.csv'
config '{"region":"us-west-1"}'
credentials '{"aws_access_key_id": "",
            "aws_secret_access_key": ""}'
skip duplicate key errors
into table `farfetch_listings`
format csv
fields terminated by ','
lines terminated by '\n';

In [ ]:
%%sql
start pipeline farfetch_pipeline;

In [ ]:
%%sql
select DATABASE_NAME, PIPELINE_NAME, BATCH_ID, BATCH_STATE, START_TIME, ROWS_STREAMED, ROWS_PER_SEC
from information_schema.PIPELINES_BATCHES_SUMMARY

In [ ]:
%%sql
select * from farfetch_listings limit 3;

## Step 3: Embed the images

In [ ]:
pip install fashion-clip

In [ ]:
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import sqlalchemy as sa
import json

In [ ]:
fclip = FashionCLIP('fashion-clip')

In [ ]:
def safe_encode_images(fclip, model_image_url, cutout_image_url):
    try:
        # Attempt to encode both model and cutout images
        model_embedding = fclip.encode_images([model_image_url], batch_size=30)
        cutout_embedding = fclip.encode_images([cutout_image_url], batch_size=30)
        return model_embedding, cutout_embedding
    except Exception as e:
        print(f"Error encoding images: Model URL: {model_image_url}, CutOut URL: {cutout_image_url}. Error: {e}")
        return None, None

In [ ]:
engine = sa.create_engine(connection_url)
conn = engine.connect()
print("Connected to SingleStore!")

Read data from `farfetch_listings` table into pandas dataframe.

In [ ]:
query = sa.text('''SELECT * FROM farfetch_listings;''')
df = pd.read_sql(query, con=conn)

In [ ]:
df.to_sql('farfetch_listings', con=conn, if_exists='append', index=False)

Each product has a model image and a cutout image. We will generate embeddings for both images.

In [ ]:
for index, row in df.iloc[2:10].iterrows():
    # model_image_url, cutout_image_url = row['image_model_url'], row['image_cutout_url']
    model_embedding, cutout_embedding = safe_encode_images(fclip, row['image_model_url'], row['image_cutout_url'])
    
    if model_embedding is not None and cutout_embedding is not None:
        model_embedding_str = '[' + ','.join(map(str, model_embedding.flatten().tolist())) + ']'
        cutout_embedding_str = '[' + ','.join(map(str, cutout_embedding.flatten().tolist())) + ']'

        df.at[index, 'model_embedding'] = json.dumps(model_embedding_str)
        df.at[index, 'cutout_embedding'] = json.dumps(cutout_embedding_str)        

## Step 4: Add e columns to table to store the embeddings

In [ ]:
%%sql
ALTER TABLE farfetch_listings ADD COLUMN `model_embedding` vector(512, F32);
ALTER TABLE farfetch_listings ADD COLUMN `cutout_embedding` vector(512, F32);

In [ ]:
%%sql
# export df to a temp table
df.to_sql('temp', con=conn, if_exists='replace', index=False)

In [ ]:
%%sql
# join the embeddings columns onto our table
UPDATE farfetch_listings
JOIN temp
ON farfetch_listings.image_cutout_url = temp.image_cutout_url
SET farfetch_listings.model_embedding = temp.model_embedding,
    farfetch_listings.cutout_embedding = temp.cutout_embedding;